In [1]:
# Set up Account Access Key
# spark.conf.set(
#  "fs.azure.account.key.<storage-account-name>.blob.core.windows.net",
#  "<storage-access-key>")

spark.conf.set(
  "fs.azure.account.key.dbblobstore.blob.core.windows.net",
  "VHiGGTYmDtROgk+821m0fHqowvikN5WTQtK51KQQz/ME9dokZCrvVjYpCnT7WgYF1o6Xm+4RIu+Omhz1bzTF2Q==")

In [2]:
#Access files in your container as if they were local files
#dbutils.fs.ls("wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>")
dbutils.fs.ls("wasbs://dbdemo01@dbblobstore.blob.core.windows.net")

In [3]:
# Mount a Blob storage container or a folder inside a container
# dbutils.fs.mount(
#   source = "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>",
#   mount_point = "<mount-point-path>",
#   extra_configs = <"<conf-key>": "<conf-value>">)
# [note] <mount_point> is a DBFS path and the path must be under /mnt

dbutils.fs.mount(
  source = "wasbs://dbdemo01@dbblobstore.blob.core.windows.net",
  mount_point = "/mnt/dbdemo01",
  extra_configs = {"fs.azure.account.key.dbblobstore.blob.core.windows.net": "VHiGGTYmDtROgk+821m0fHqowvikN5WTQtK51KQQz/ME9dokZCrvVjYpCnT7WgYF1o6Xm+4RIu+Omhz1bzTF2Q=="})

In [4]:
dbutils.fs.ls("/mnt/dbdemo01")

In [5]:
# Access files in your container as if they were local files
# (TEXT) df = spark.read.text("/mnt/%s/...." % <mount-point-path>)
# (JSON) df = spark.read.json("/mnt/%s/...." % <mount-point-path>)

df = spark.read.json( "/mnt/%s/small_radio_json.json" % "dbdemo01")

# display(df)
df.show()

In [6]:
specificColumnsDf = df.select("firstname", "lastname", "gender", "location", "level")
specificColumnsDf.show()

In [7]:
renamedColumnsDF = specificColumnsDf.withColumnRenamed("level", "subscription_type")
renamedColumnsDF.show()

In [8]:
#Load data into Azure Cosmos DB
#writeConfig = {
# "Endpoint" : "https://<cosmosdb-account-name>.documents.azure.com:443/",
# "Masterkey" : "<Cosmosdb-master-key-string>",
# "Database" : "<database-name>",
# "Collection" : "<collection-name>",
# "Upsert" : "true"
#}

# Write configuration
writeConfig = {
 "Endpoint" : "https://mysqldb.documents.azure.com:443/",
 "Masterkey" : "ZLLIuOgqdWwORZWP44coCCokCfrJ0bLdrSw2gxz62UYkd3IYgdCO5EvbUoJ4soZ8zH2btRmyaXw57tL1SDSzxg==",
 "Database" : "samples",
 "Collection" : "radiosample",
 "Upsert" : "true"
}

# Write to Cosmos DB from the renamedColumnsDF DataFrame
renamedColumnsDF.write.format("com.microsoft.azure.cosmosdb.spark").options(**writeConfig).save()